# Sklearn Pipeline Permuter Example

In [ ]:
import pandas as pd
import numpy as np

# Utils
from sklearn.base import clone

# Preprocessing & Feature Selection
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# Cross-Validation
from sklearn.model_selection import KFold

from biopsykit.classification.model_selection import SklearnPipelinePermuter

%load_ext autoreload
%autoreload 2

In [ ]:
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

In [ ]:
model_dict = {
    "scaler": {
        "StandardScaler": StandardScaler(),
        "MinMaxScaler": MinMaxScaler()
    },
    "reduce_dim": {
        "SelectKBest": SelectKBest(),
        "RFE": RFE(SVC(kernel="linear", C=1))
    },
    "clf" : {
        "KNeighborsClassifier": KNeighborsClassifier(),
        "DecisionTreeClassifier": DecisionTreeClassifier(),
        "SVC": SVC(),
        "AdaBoostClassifier": AdaBoostClassifier(),
    }
}

In [ ]:
C_OPTIONS = np.logspace(start=-3, stop=3, num=7)
GAMMA_OPTIONS = np.logspace(start=-3, stop=3, num=7)

params_dict = {
    "StandardScaler": None,
    "MinMaxScaler": None,
    "SelectKBest": { "k": [2, 4, "all"] },
    "RFE": { "n_features_to_select": [2, 4, None] },
    "KNeighborsClassifier": { "n_neighbors": [2, 4], "weights": ["uniform", "distance"] },
    "DecisionTreeClassifier": {"criterion": ['gini', 'entropy'], "max_depth": [2, 4] },
    "AdaBoostClassifier": {
        "base_estimator": [DecisionTreeClassifier(max_depth=1)],
        "n_estimators": np.arange(20, 110, 10),
        "learning_rate": np.arange(0.6, 1.1, 0.1)
    },
    "SVC": [
        {
            "kernel": ["linear"],
            "C": np.logspace(start=-2, stop=2, num=5)
        },
        {
            "kernel": ["rbf"],
            "C": np.logspace(start=-2, stop=2, num=5),
            "gamma": np.logspace(start=-2, stop=2, num=5)
        }
    ]
}

In [ ]:
pipeline_permuter = SklearnPipelinePermuter(model_dict, params_dict)

In [ ]:
outer_cv = KFold(5)
inner_cv = KFold(5)

pipeline_permuter.fit(X, y, outer_cv=outer_cv, inner_cv=inner_cv)

In [ ]:
pipeline_permuter.pipeline_score_results()

In [ ]:
pipeline_permuter.mean_pipeline_score_results()

In [ ]:
pipeline_permuter.best_pipeline()

In [ ]:
pipeline_permuter.metric_summary()